### load Lucile evaluation data and average

In [ ]:
###### import numpy as np
from os.path import join
import pandas as pd
from itertools import combinations
import numpy as np

#MICP is not included (only his memory perf can be used, no RT data)
PATH = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/'
df_name = join(PATH,'EvalsOdeurs_Lucile.xlsx')
df_od_AL = join(PATH, 'Recap_Odeurs_Evaluations.xlsx')
df_save = join(PATH, 'Eval_Odors_avg.csv')

studies = ['Etude1.1','Etude1.2','Etude2']
col_sel = ['PARTICIPANT','Stimulus','hedo_scaled','fam_scaled','emo_scaled']

df_tot = pd.DataFrame([])
for stud in studies:
    df = pd.read_excel(df_name,sheet_name=stud)
    df_sel = df[col_sel].dropna()
    df_tot = df_tot.append(df_sel)
df_tot.to_csv(df_save, columns=col_sel)
#get a consistent odor name
df_tot = df_tot.replace('Basilic Comores', 'Basilic comores')
df_tot = df_tot.replace('Acétate Linalyle', 'Acétate linalyle')

df_su = df_tot.groupby(['Stimulus']).agg({'PARTICIPANT' : 'count', 'hedo_scaled':np.mean,
                                         'fam_scaled':'mean', 'emo_scaled':'mean'})
print(df_su)
print('#number of participants in total',len(np.unique(df_tot[['PARTICIPANT']])))
print('>>> average nb of participants', np.mean(df_su[['PARTICIPANT']].values))


In [ ]:
from scipy.stats import pearsonr
A = [2.67,6.32,2.86,4.76,3.82,3.73,5.86,4.00,2.86,2.62,5.19,2.80,3.38,4.27,5.83,4.66,6.43,4.96]
B = [4.91,5.64,4.54,4.62,5.2,4.19237472766885,5.91401597676108,4.9,4.58,4.67,7.66,5.25,4.18,4.48,4.84684095860566,5.73,6.78,4.47671194839592,]
R,p = pearsonr(A,B)
print(R,p)

In [ ]:
from scipy import stats
def tpsim_btw_2odors(pow_o1,pow_o2,stat='pearson',average=True):
    """
    Compute tpsim between 2 odors 
    Parameters
    ----------
    pow_o1, pow_o2 : array (npts x ntrials)
    stat : string
            The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : array (Average TPSim between 2 ODORS)
    """
    
    corr = stats.pearsonr if stat == 'pearson' else stats.spearmanr
    sim_trials = np.array([])
    ncombs = len([a for a,_ in product(range(pow_o1.shape[-1]),range(pow_o2.shape[-1]))])
    for t0, t1 in product(range(pow_o1.shape[-1]),range(pow_o2.shape[-1])):
        R, _ = corr(pow_o1[:,t0],pow_o2[:,t1])
        sim_trials = np.vstack((sim_trials,1-R)) if np.size(sim_trials) else np.array(1-R)
    if ncombs == 1:
        sim_trials = np.array(sim_trials)[np.newaxis][np.newaxis]
    if average == True:
        sim_trials = np.mean(sim_trials)
    return sim_trials

In [ ]:
from brainpipe.system import study
from os.path import join, exists
from os import makedirs
import pandas as pd
from itertools import product, combinations
import numpy as np
from utils import odor_su_score
"""
Compute similarity btw all odors by subject for all electrodes
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs_EL.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}/similarity_matrix_btw_v=1_elecs=all_early_late/')
tps_save = join(path2save, 'TPS_{}_{}_{}_btw_odors_mean={}_sheet_{}_late.npz')

PATH = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/'
df_od_avg = join(PATH, 'Recap_Odeurs_Evaluations.xlsx')
###############################################################################
exp, mean = 'Enc', False#Ret, Enc
stat = 'pearson'
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0],exp[0]]
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs, f_id = ['theta'], 1
odors_su = {'CHAF': [1,2,4,5,3,8,7,9], #low than high
            'LEFC': [15,2,1,16,14,3,4,17],
            'PIRJ': [1,9,5,4,6,7,18], #missing odor 15
            'VACJ': [11,14,12,10,15,17,16,13],
            'SEMC': [7,10,11,12,13,5,8,9],
            'FERJ': [7,2,16,17,12,1,5,13]}

sheet = 'Final' #,'Final_Lucile','Final_avg']
#Final (just 2 odors added from Lucile) Final Lucile (all data available from Lucile taken)
#Final avg (Lucile's and my data are averaged when possible)
col_sel = ['od_num','odors','Pleasantness','Familiarity']
dims = ['Pleasantness','Familiarity']

df = pd.read_excel(df_od_avg,sheet_name=sheet)

for su,freq in product(odors_su,freqs):
    ncomb = len([o1+o2 for o1,o2 in combinations(odors_su[su],2)])
    mat_ = np.load(pow_path.format(su,str(odors_su[su][0]),shortcut[0]),allow_pickle=True)
    chans, labels, xyz = mat_['channels'], mat_['labels'], mat_['xyz']
    nelecs = len(chans)
    tps_list = np.array([])
    print(ncomb,nelecs)
    list_comb, dist_pleas, dist_fam = np.array([]),np.array([]),np.array([])
    dist_pleas_fam, dist_sc_pl, dist_sc_fam = np.array([]), np.array([]), np.array([])
    
    for o1, o2 in combinations(odors_su[su],2):
        mat_o1 = np.load(pow_path.format(su,str(o1),shortcut[0]))
        mat_o2 = np.load(pow_path.format(su,str(o2),shortcut[1]))
        if (mat_o1['xpow_L'].shape[0] > 0) & (mat_o2['xpow_L'].shape[0] > 0):
            pow_o1 = mat_o1['xpow_L'][f_id,:,17:47,:] #nelecs,npts,ntrials
            pow_o2 = mat_o2['xpow_L'][f_id,:,17:47,:] #nelecs,npts,ntrials
            nvals = len([a for a,b in product(range(pow_o1.shape[-1]), range(pow_o2.shape[-1]))])
            pleas_o1 = df.loc[df['od_num']=='O'+str(o1)].iloc[:,2].values
            fam_o1 = df.loc[df['od_num']=='O'+str(o1)].iloc[:,3].values
            pleas_o2 = df.loc[df['od_num']=='O'+str(o2)].iloc[:,2].values
            fam_o2 = df.loc[df['od_num']=='O'+str(o2)].iloc[:,3].values
            sc_o1, sc_o2 = [odor_su_score[su][o1]], [odor_su_score[su][o2]]
            a, b = np.array([pleas_o1,fam_o1]), np.array([pleas_o2,fam_o2])
            c, d = np.array([pleas_o1,fam_o1,sc_o1]), np.array([pleas_o2,fam_o2,sc_o2])
            e, f = np.array([fam_o1,sc_o1]), np.array([fam_o2,sc_o2])
            dist_pl = np.round(np.linalg.norm(pleas_o1-pleas_o2),2)
            dist_pl_fam = np.round(np.linalg.norm(a-b),2)
            dist_f = np.round(np.linalg.norm(fam_o1-fam_o2),2)
            dist_score_pl = np.round(np.linalg.norm(c-d),2)
            dist_score_fam = np.round(np.linalg.norm(e-f),2)

            if mean == False:
                list_comb = np.hstack((list_comb,[str(o1)+'_'+str(o2)]*nvals)) if np.size(list_comb) else [str(o1)+'_'+str(o2)]*nvals
                dist_pleas = np.hstack((dist_pleas,[dist_pl]*nvals)) if np.size(dist_pleas) else [dist_pl]*nvals 
                dist_pleas_fam = np.hstack((dist_pleas_fam,[dist_pl_fam]*nvals)) if np.size(dist_pleas_fam) else [dist_pl_fam]*nvals 
                dist_fam = np.hstack((dist_fam,[dist_f]*nvals)) if np.size(dist_fam) else [dist_f]*nvals
                dist_sc_pl = np.hstack((dist_sc_pl,[dist_score_pl]*nvals)) if np.size(dist_sc_pl) else [dist_score_pl]*nvals
                dist_sc_fam = np.hstack((dist_sc_fam,[dist_score_fam]*nvals)) if np.size(dist_sc_fam) else [dist_score_fam]*nvals
            else: 
                dist_pleas = np.hstack((dist_pleas,dist_pl)) if np.size(dist_pleas) else dist_pl
                dist_fam = np.hstack((dist_fam,dist_f)) if np.size(dist_fam) else dist_f
                list_comb = np.hstack((list_comb,str(o1)+'_'+str(o2))) if np.size(list_comb) else str(o1)+'_'+str(o2)
        
            od_tps = np.array([])   
            for elec in range(nelecs):
                tps_ = tpsim_btw_2odors(pow_o1[elec],pow_o2[elec],
                                                    stat=stat,average=mean).swapaxes(0,1)
                od_tps = np.concatenate((od_tps,tps_),axis=0) if np.size(od_tps) else tps_
            tps_list = np.concatenate((tps_list,od_tps),axis=1) if np.size(tps_list) else od_tps
    print(tps_list.shape,list_comb.shape,dist_fam.shape,dist_pleas.shape,dist_sc_pl.shape)
    np.savez(tps_save.format(exp,stat,su,freq,mean,sheet),tps=tps_list,comb=list_comb,pleas=dist_pleas,
                fam=dist_fam,channels=chans,labels=labels, xyz=xyz,pl_fam=dist_pleas_fam,
            sc_pl=dist_sc_pl, sc_fam= dist_sc_fam)

In [ ]:
from brainpipe.system import study
from os.path import join, exists
from os import makedirs
import pandas as pd
from itertools import product
import numpy as np
from utils import odor_groups_3wgth
"""
Compute similarity btw LOW and HIGH MEM GROUPS by subject for all electrodes
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}/similarity_matrix_btw_v=1_elecs=all/')
tps_save = join(path2save, 'TPS_{}_{}_{}_{}_btw_odors_mean={}_sheet_{}.npz')

PATH = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/'
df_od_avg = join(PATH, 'Recap_Odeurs_Evaluations.xlsx')
###############################################################################
exp, mean = 'Enc', False#Ret, Enc
stat = 'pearson'
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0],exp[0]]
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs, f = ['theta'], 1
sheet = 'Final_Lucile' #,'Final_Lucile','Final_avg']
#Final (just 2 odors added from Lucile) Final Lucile (all data available from Lucile taken)
#Final avg (Lucile's and my data are averaged when possible)
col_sel = ['od_num','odors','Pleasantness','Familiarity']
conds = ['low','mid','high']
dims = ['Pleasantness','Familiarity']

df = pd.read_excel(df_od_avg,sheet_name=sheet)

for su,freq in product(odor_groups_3wgth,freqs):
    for cond in conds:
        for od in odor_groups_3wgth[su][cond]:
            print(su,cond,od)
            ncomb = len([o1+o2 for o1,o2 in combinations(odors_su[su],2)])
            mat_ = np.load(pow_path.format(su,str(odors_su[su][0]),shortcut[0]),allow_pickle=True)
            chans, labels, xyz = mat_['channels'], mat_['labels'], mat_['xyz']
            nelecs = len(chans)
            tps_list = np.array([])
            print(ncomb,nelecs)
            list_comb, dist_pleas, dist_fam = np.array([]),np.array([]),np.array([])
            dist_pleas_fam = np.array([])
    
            for o1, o2 in combinations(odor_groups_3wgth[su][cond],2):
                mat_o1 = np.load(pow_path.format(su,str(o1),shortcut[0]))
                pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
                mat_o2 = np.load(pow_path.format(su,str(o2),shortcut[1]))
                pow_o2 = mat_o2['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
                nvals = len([a for a,b in product(range(pow_o1.shape[-1]), range(pow_o2.shape[-1]))])
                print(pow_o1.shape, pow_o2.shape)
                pleas_o1 = df.loc[df['od_num']=='O'+str(o1)].iloc[:,2].values
                fam_o1 = df.loc[df['od_num']=='O'+str(o1)].iloc[:,3].values
                pleas_o2 = df.loc[df['od_num']=='O'+str(o2)].iloc[:,2].values
                fam_o2 = df.loc[df['od_num']=='O'+str(o2)].iloc[:,3].values
                a, b = np.array([pleas_o1,fam_o1]), np.array([pleas_o2,fam_o2])
                dist_pl = np.round(np.linalg.norm(pleas_o1-pleas_o2),2)
                dist_pl_fam = np.round(np.linalg.norm(a-b),2)
                dist_f = np.round(np.linalg.norm(fam_o1-fam_o2),2)

                if mean == False:
                    list_comb = np.hstack((list_comb,[str(o1)+'_'+str(o2)]*nvals)) if np.size(list_comb) else np.array([str(o1)+'_'+str(o2)]*nvals)
                    dist_pleas = np.hstack((dist_pleas,[dist_pl]*nvals)) if np.size(dist_pleas) else np.array([dist_pl]*nvals )
                    dist_pleas_fam = np.hstack((dist_pleas_fam,[dist_pl_fam]*nvals)) if np.size(dist_pleas_fam) else np.array([dist_pl_fam]*nvals )
                    dist_fam = np.hstack((dist_fam,[dist_f]*nvals)) if np.size(dist_fam) else np.array([dist_f]*nvals)
                else: 
                    dist_pleas = np.hstack((dist_pleas,dist_pl)) if np.size(dist_pleas) else dist_pl
                    dist_fam = np.hstack((dist_fam,dist_f)) if np.size(dist_fam) else dist_f
                    list_comb = np.hstack((list_comb,str(o1)+'_'+str(o2))) if np.size(list_comb) else str(o1)+'_'+str(o2)

                od_tps = np.array([])   
                for elec in range(nelecs):
                    tps_ = tpsim_btw_2odors(pow_o1[elec],pow_o2[elec],
                                                        stat=stat,average=mean).swapaxes(0,1)
                    od_tps = np.concatenate((od_tps,tps_),axis=0) if np.size(od_tps) else tps_
                tps_list = np.concatenate((tps_list,od_tps),axis=1) if np.size(tps_list) else od_tps
            print(tps_list.shape,list_comb.shape,dist_fam.shape,dist_pleas.shape)
            np.savez(tps_save.format(exp,stat,su,cond,freq,mean,sheet),tps=tps_list,comb=list_comb,pleas=dist_pleas,
                        fam=dist_fam,channels=chans,labels=labels, xyz=xyz,pl_fam=dist_pleas_fam)

### correlate TPSim and odor characteristics

In [ ]:
from utils import odor_groups_3wgth, rename_elecs
from os.path import join, exists
from os import makedirs
from scipy.stats import pearsonr, kendalltau
from mne.stats import bonferroni_correction, fdr_correction

st = study('Olfacto')
exp = 'Enc'#'Enc'
freqs = ['theta'] #freqname in filename
path_pow = join(st.path, 'feature/TPSim_3groups_'+exp+ \
                        '/similarity_matrix_btw_v=1_elecs=all_early_late/')
filename = join(path_pow, 'TPS_pearson_{}_{}_btw_odors_mean=False_sheet_{}_late.npz')
df_name = join(path_pow, '{}_correl_rdm_{}_{}_mean=False_{}_L.csv') #su, conds0, conds1, freq
rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf','OFC_olf','SFG']
sheets = ['Final']
# cond = 'high'

for sheet in sheets:
    for freq in freqs:
        subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
        channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
        R_vals_pl, R_vals_fam, R_vals_pl_fam = np.array([]), np.array([]), np.array([])
        R_vals_sc_pl, R_vals_sc_fam = np.array([]), np.array([])
        p_vals_pl, ps_fdr_pl, ps_bf_pl = np.array([]), np.array([]), np.array([])
        p_vals_fam, ps_fdr_fam, ps_bf_fam = np.array([]), np.array([]), np.array([])
        p_vals_pl_fam, ps_fdr_pl_fam, ps_bf_pl_fam = np.array([]), np.array([]), np.array([])
        p_vals_sc_pl, ps_fdr_sc_pl, ps_bf_sc_pl = np.array([]), np.array([]), np.array([])
        p_vals_sc_fam, ps_fdr_sc_fam, ps_bf_sc_fam = np.array([]), np.array([]), np.array([])

        for su in odor_groups_3wgth:
            #load similarity btw odors and distance pleasantness and familiarity
            # ['tps', 'comb', 'pleas', 'fam', 'channels', 'labels', 'xyz']
            mat0 = np.load(filename.format(su,freq,sheet),allow_pickle=True)

            #select electrodes
            labels, channels = mat0['labels'], mat0['channels']
            x, y, z = mat0['xyz'][:,0], mat0['xyz'][:,1], mat0['xyz'][:,2]
            labels_new = rename_elecs(labels,x,y,z)
            idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
            nelecs = len(idx_sel)
            print(su,nelecs,mat0['tps'].shape,mat0['channels'].shape)
            labels, channels = labels_new[idx_sel], channels[idx_sel]
            x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]

            tps_sim, pleas, fam = mat0['tps'], np.ravel(mat0['pleas']), np.ravel(mat0['fam'])
            pl_fam, sc_pl, sc_fam = mat0['pl_fam'], mat0['sc_pl'], mat0['sc_fam']
            print(su,freq,'pl',pleas.shape,'fam',fam.shape,'pl_fam',pl_fam.shape)

            #compute stats // correlate distance btw odors
            R_pl, unc_p_pl, R_fam, unc_p_fam = [], [], [], []
            R_pl_fam, unc_pl_fam = [], []
            R_sc_pl, unc_sc_pl, R_sc_fam, unc_sc_fam = [], [],[], []
            for elec in range(nelecs):
                R1, p1 = kendalltau(tps_sim[elec],pleas)
                R3, p3 = kendalltau(tps_sim[elec],pl_fam)
                R2, p2 = kendalltau(tps_sim[elec],fam)
                R4, p4 = kendalltau(tps_sim[elec],sc_pl)
                R5, p5 = kendalltau(tps_sim[elec],sc_fam)
                R_pl.append(R1), unc_p_pl.append(p1)
                R_fam.append(R2), unc_p_fam.append(p2)
                R_pl_fam.append(R3), unc_pl_fam.append(p3)
                R_sc_pl.append(R4), unc_sc_pl.append(p4)
                R_sc_fam.append(R5), unc_sc_fam.append(p5)
            _, p_fdr_pl = fdr_correction(unc_p_pl)
            _, p_fdr_fam = fdr_correction(unc_p_fam)
            _, p_fdr_pl_fam = fdr_correction(unc_pl_fam)
            _, p_fdr_sc_pl = fdr_correction(unc_sc_pl)
            _, p_fdr_sc_fam = fdr_correction(unc_sc_fam)
            
            _, p_bf_pl = bonferroni_correction(unc_p_pl)
            _, p_bf_fam = bonferroni_correction(unc_p_fam)
            _, p_bf_pl_fam = bonferroni_correction(unc_pl_fam)
            _, p_bf_sc_fam = bonferroni_correction(unc_sc_fam)
            _, p_bf_sc_pl = bonferroni_correction(unc_sc_pl)

            #Fill the csv file with elec infos and stats
            subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
            elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
            labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
            channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
            x_c = np.hstack((x_c,x)) if np.size(x_c) else x
            y_c = np.hstack((y_c,y)) if np.size(y_c) else y
            z_c = np.hstack((z_c,z)) if np.size(z_c) else z

            R_vals_pl = np.hstack((R_vals_pl,R_pl)) if np.size(R_vals_pl) else R_pl
            R_vals_fam = np.hstack((R_vals_fam,R_fam)) if np.size(R_vals_fam) else R_fam
            R_vals_pl_fam = np.hstack((R_vals_pl_fam,R_pl_fam)) if np.size(R_vals_pl_fam) else R_pl_fam
            R_vals_sc_fam = np.hstack((R_vals_sc_fam,R_sc_fam)) if np.size(R_vals_sc_fam) else R_sc_fam
            R_vals_sc_pl = np.hstack((R_vals_sc_pl,R_sc_pl)) if np.size(R_vals_sc_pl) else R_sc_pl

            p_vals_pl = np.hstack((p_vals_pl,unc_p_pl)) if np.size(p_vals_pl) else unc_p_pl
            p_vals_fam = np.hstack((p_vals_fam,unc_p_fam)) if np.size(p_vals_fam) else unc_p_fam
            p_vals_pl_fam = np.hstack((p_vals_pl_fam,unc_pl_fam)) if np.size(p_vals_pl_fam) else unc_pl_fam
            p_vals_sc_fam = np.hstack((p_vals_sc_fam,unc_sc_fam)) if np.size(p_vals_sc_fam) else unc_sc_fam
            p_vals_sc_pl = np.hstack((p_vals_sc_pl,unc_sc_pl)) if np.size(p_vals_sc_pl) else unc_sc_pl

            ps_fdr_pl = np.hstack((ps_fdr_pl,p_fdr_pl)) if np.size(ps_fdr_pl) else p_fdr_pl
            ps_fdr_fam = np.hstack((ps_fdr_fam,p_fdr_fam)) if np.size(ps_fdr_fam) else p_fdr_fam
            ps_fdr_pl_fam = np.hstack((ps_fdr_pl_fam,p_fdr_pl_fam)) if np.size(ps_fdr_pl_fam) else p_fdr_pl_fam
            ps_fdr_sc_fam = np.hstack((ps_fdr_sc_fam,p_fdr_sc_fam)) if np.size(ps_fdr_sc_fam) else p_fdr_sc_fam
            ps_fdr_sc_pl = np.hstack((ps_fdr_sc_pl,p_fdr_sc_pl)) if np.size(ps_fdr_sc_pl) else p_fdr_sc_pl

            ps_bf_pl= np.hstack((ps_bf_pl,p_bf_pl)) if np.size(ps_bf_pl) else p_bf_pl
            ps_bf_fam = np.hstack((ps_bf_fam,p_bf_fam)) if np.size(ps_bf_fam) else p_bf_fam
            ps_bf_pl_fam = np.hstack((ps_bf_pl_fam,p_bf_pl_fam)) if np.size(ps_bf_pl_fam) else p_bf_pl_fam
            ps_bf_sc_fam = np.hstack((ps_bf_sc_fam,p_bf_sc_fam)) if np.size(ps_bf_sc_fam) else p_bf_sc_fam
            ps_bf_sc_pl = np.hstack((ps_bf_sc_pl,p_bf_sc_pl)) if np.size(ps_bf_sc_pl) else p_bf_sc_pl

        data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                    channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                    z_c[:,np.newaxis],elecs_c[:,np.newaxis],
                    R_vals_pl[:,np.newaxis],p_vals_pl[:,np.newaxis],ps_fdr_pl[:,np.newaxis],
                               ps_bf_pl[:,np.newaxis],
                    R_vals_fam[:,np.newaxis],p_vals_fam[:,np.newaxis],ps_fdr_fam[:,np.newaxis],
                               ps_bf_fam[:,np.newaxis],
                    R_vals_pl_fam[:,np.newaxis],p_vals_pl_fam[:,np.newaxis],ps_fdr_pl_fam[:,np.newaxis],
                               ps_bf_pl_fam[:,np.newaxis],
                    R_vals_sc_fam[:,np.newaxis],p_vals_sc_fam[:,np.newaxis],ps_fdr_sc_fam[:,np.newaxis],
                               ps_bf_sc_fam[:,np.newaxis],
                    R_vals_sc_pl[:,np.newaxis],p_vals_sc_pl[:,np.newaxis],ps_fdr_sc_pl[:,np.newaxis],
                               ps_bf_sc_pl[:,np.newaxis]),axis=1)
        df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z','elecs_num',
                    'R_pl','unc_pl','fdr_pl','bonf_pl','R_fam','unc_fam','fdr_fam','bonf_fam',
                    'R_pl_fam','unc_pl_fam','fdr_pl_fam','bonf_pl_fam',
                    'R_sc_fam','unc_sc_fam','fdr_sc_fam','bonf_sc_fam',
                    'R_sc_pl','unc_sc_pl','fdr_sc_pl','bonf_sc_pl'])
        print(df.shape)
        df.to_csv(df_name.format('All_subjects','pleas_fam_score',freq,sheet),index=False)           


### summarize stats 

In [ ]:
from brainpipe.system import study
from collections import Counter
from os import path
import pandas as pd
from itertools import product
import numpy as np

st = study('Olfacto')
fold, freq, sheet = 'Enc', 'theta', 'Final'
olf_regions = ['Amg','pPirT','OFC_olf','Ins_olf']
# cond = 'high'
path_file = path.join(st.path, 'feature/TPSim_3groups_'+fold+\
                      '/similarity_matrix_btw_v=1_elecs=all_early_late/')
df_name = path.join(path_file, 
            'All_subjects_correl_rdm_pleas_fam_score_'+freq+'_mean=False_'+sheet+'_L.csv') #su, conds0, conds1, freq
df_save = path.join(path_file, 
                'All_subjects_correl_{}_{}_'+freq+'_'+sheet+'_mean=False_{}_L.csv')

thrs = [0.05]
corrections = ['fdr_']#['fdr_','bonf_']
dims = ['pl','fam','pl_fam']
# dims = ['pl_fam']

df_init = pd.read_csv(df_name)
print('Initial df shape', df_init.shape,df_init.columns)

for th, dim, corr in product(thrs,dims,corrections):
    df_sel = df_init.loc[df_init[corr+dim]<th]
    df_sel['sign'] = ['similar' if t > 0 else 'different' for t in df_sel['R_'+dim]]
    print('\n stats at p < ',th, 'correction : ',corr, df_sel.shape, 'for dimension',dim)
    print(Counter(df_sel['labels']))

    rois = np.unique(df_sel['labels'])
    for roi in rois:
        df_roi = df_sel.loc[df_sel['labels']==roi]
        df_inc = df_roi.loc[df_roi['sign']=='different'].groupby(['subjects']).count()
        df_dec = df_roi.loc[df_roi['sign']=='similar'].groupby(['subjects']).count()
#         if roi in ['pPirT','Amg']:
#             print(df_roi)
        if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in olf_regions): # we only consider positive correlations
            print(roi, 'NB of subjects with SIMILAR',df_dec.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='similar']
            print('#electrodes in total >>> ',df_plot.shape[0])
#             df_plot.to_csv(df_save.format(roi,corr+str(th),dim,cond))
#             print(df_plot)


In [ ]:
"""Only consider specific ROIs and not ALL brain regions"""
from brainpipe.system import study
from mne.stats import bonferroni_correction, fdr_correction
from collections import Counter
from os import path
import pandas as pd
from itertools import product
import numpy as np
import math

st = study('Olfacto')
fold, freq, sheet = 'Enc', 'theta', 'Final'
# cond = 'high'
path_file = path.join(st.path, 
          'feature/TPSim_3groups_'+fold+'/similarity_matrix_btw_v=1_elecs=all_early_late/')
df_name = path.join(path_file, 
                'All_subjects_correl_rdm_pleas_fam_score_'+freq+'_mean=False_'+sheet+'_E.csv') #su, conds0, conds1, freq
df_save = path.join(path_file, 
                'All_subjects_correl_{}_{}_{}_'+freq+'_'+sheet+'_mean=False_{}_E.csv')

thrs = [0.05]
corrections = ['unc_']#['fdr_','bonf_']
dims = ['pl','fam']
rois = ['OFC_olf','pPirT']
olf_regions = ['OFC_olf','pPirT']

df_init = pd.read_csv(df_name)
#print('Initial df shape', df_init.shape,df_init.columns)
# combine Amg/pPirT together
df_init['labels'] = [x if x != 'Amg' else 'pPirT' for x in df_init['labels']]

for th, dim, corr in product(thrs,dims,corrections):
    for roi in rois:
        print('>>> processing', roi, dim, corr, th)
        df_roi = df_init.loc[df_init['labels']==roi]
        pvals = [p if not math.isnan(p) else 1 for p in df_roi['unc_'+dim].values]
        df_roi['new_pvalues'] = fdr_correction(pvals)[1]
        #print(df_roi[['subjects','labels','channels','R_'+dim]].mean())
        #print(df_roi[['subjects','labels','channels','R_'+dim]].sem())
        #             'unc_'+dim, 'fdr_'+dim, 'new_pvalues']].loc[df_roi['unc_'+dim]<th])
        df_sel = df_roi.loc[df_roi['new_pvalues']<th]
        print('sig results',df_sel)
        df_sel['sign'] = ['similar' if t > 0 else 'different' for t in df_sel['R_'+dim]]
        print('\n stats at p < ',th, 'correction : ',corr, df_sel.shape, 'for dimension',dim)
        print(Counter(df_sel['labels']))

        df_dec = df_sel.loc[df_sel['sign']=='similar'].groupby(['subjects']).count()
        if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in olf_regions):
            print(roi, 'NB of subjects with SIMILAR',df_dec.shape[0],' subjects')
            df_plot = df_sel.loc[df_sel['sign']=='similar']
            print('#electrodes in total >>> ',df_plot.shape[0])
            df_plot[['subjects','labels','channels','x','y','z','R_'+dim,
                         'unc_'+dim, 'fdr_'+dim,'new_pvalues']].to_csv(df_save.format(roi,corr+str(th),'late',dim))
            print(df_plot[['subjects','labels','channels','x','y','z','R_'+dim,
                         'unc_'+dim, 'fdr_'+dim,'new_pvalues']])
            print(df_plot['R_'+dim].mean(), df_plot['R_'+dim].std())
            


In [ ]:
"""
//// PLEASANTNESS + FAMILIARITY ////
Final OFC (2 patients 3 elecs)
Final avg OFC (3 patients 5 elecs) + MFG (3 patients 4 elecs)

//// FAMILIARITY ////
Final OFC (2P,3elecs), MFG (3P, 5elecs)
Final avg MFG (3P,6elecs), OFC (2P 2elecs 2 sens), aHC (3P,4elecs)

//// PLEASANTNESS ////
Final OFC (3P, 4elecs)
Final avg OFC (3P, 4elecs)
"""

In [ ]:
from os.path import join
from brainpipe.system import study
import pandas as pd
import numpy as np
from itertools import combinations

st = study('Olfacto')
PATH = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/'
filename = 'Recap_Odeurs_Evaluations.xlsx'
PATH_SAVE = join(st.path, 'feature/TPSim_3groups_Enc/similarity_matrix_btw/')
savename = 'dist_pl_fam_all_odors_pleas.npz'

df = pd.read_excel(PATH+filename, sheet_name='Final')
odors = df[['od_num']].values[:,0]
#odors = [14,10,3,8,13,6,12,9,1,18,4,15,5,17,11,2,16,7] #ordered by Fam
odors = [10,1,12,3,9,13,6,5,8,14,16,4,18,11,15,7,2,17] #ordered by Pleas
#odors = [10,3,9,1,13,12,6,14,8,5,4,18,16,15,7,2,11,17] #fam * pleas
#odors = [10,3,9,13,1,6,12,14,8,5,4,18,16,15,7,2,11,17] #fam + pleas

print(df.columns)

dist_f, dist_p, dist_fp = [], [], []
for o1,o2 in combinations(odors,2):
    f1 = df.loc[df['od_num']=='O'+str(o1)]['Familiarity'].values
    f2 = df.loc[df['od_num']=='O'+str(o2)]['Familiarity'].values
    p1 = df.loc[df['od_num']=='O'+str(o1)]['Pleasantness'].values
    p2 = df.loc[df['od_num']=='O'+str(o2)]['Pleasantness'].values
    fp1, fp2 = np.array([f1,p1]), np.array([f2,p2])
    dist_f.append(np.round(np.linalg.norm(f1-f2),2))
    dist_p.append(np.round(np.linalg.norm(p1-p2),2))
    dist_fp.append(np.round(np.linalg.norm(fp1-fp2),2))
print(len(dist_f))
np.savez(PATH_SAVE+savename,d_f=np.array(dist_f),d_p=np.array(dist_p),
        d_fp=np.array(dist_fp))

### Plot odors RDM matrices 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import cm

from matplotlib.patches import Patch
from sklearn.manifold import MDS
"""
Plot TPSim matrices (RDM) for all odors (Familiarity Pleasantness, both)
"""
###############################################################################
st = study('Olfacto')
path_npz = join(st.path,'feature/TPSim_3groups_Enc/')
path_pow = join(path_npz, 'similarity_matrix_btw/dist_pl_fam_all_odors_mult.npz')
savename = join(path_npz, 'distance_graphs/Plot_distance_{}_all_odors_mult.png')
###############################################################################
exp = 'Enc' #Ret, Enc
###############################################################################
#new_order = [10,1,12,3,9,13,6,5,8,14,16,4,18,11,15,7,2,17] #Pleas order
new_order = [10,3,9,1,13,12,6,14,8,5,4,18,16,15,7,2,11,17] #fam * pleas
#new_order = [10,3,9,13,1,6,12,14,8,5,4,18,16,15,7,2,11,17] #fam + pleas
#new_order = [14,10,3,8,13,6,12,9,1,18,4,15,5,17,11,2,16,7] #Fam order
mat = np.load(path_pow)
features = mat.files

for feat in features:
    combs = mat[feat]
    n_od = 18
    idx = list(np.arange(1,n_od+1))
    tri = np.zeros((n_od, n_od))
    tri[np.triu_indices(n_od,1)] = combs
    tri[np.tril_indices(n_od, -1)] = tri.T[np.tril_indices(n_od, -1)]

    model = MDS(n_components=2, dissimilarity='precomputed', random_state=None)
    out = model.fit_transform(tri)
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,3))
    colors = 'black'
    markers = 'o'

    for i, txt in enumerate(idx):
        ax1.scatter(out[i,0], out[i,1], c=colors, marker=markers)
        ax1.annotate('O'+str(txt), (out[i,0], out[i,1]))
    ax1.set_xlabel('component 1')
    ax1.set_ylabel('component 2')
    #ax1.axis('equal')
        
    #subplot #1 Graph 2D 
    cmap = cm.get_cmap('viridis', 30)
    mask =  np.tri(tri.shape[0], k=0) #mask upper triangle
    A = np.ma.array(tri, mask=mask) # mask out the lower triangle
    cax = ax2.imshow(A, vmin=0,vmax=3,interpolation="nearest", cmap=cmap)
    ax2.set_xticks(np.arange(n_od))
    ax2.set_yticks(np.arange(n_od))
    ax2.set_xticklabels(new_order,fontsize=11)
    ax2.set_yticklabels(new_order,fontsize=11)
    plt.colorbar(cax)
    plt.subplots_adjust(top=0.92)

    title = 'Distance btw odors in {} domaine )'.format(feat)
    fig.suptitle(title, fontsize=12)
    
    plt.savefig(savename.format(feat))
    plt.savefig(savename.format(feat).replace('.png','.pdf'))
    plt.clf()
    plt.close()


### compute distance btw all odors (GoodSenseCompagny)
- Structure (single, mixture)
- Strength (medium, high)
- Descriptors (type of odors: floral...)
- Averaged ratings (pleasantness, familiarity)

In [ ]:
import pandas as pd
from scipy.stats import pearsonr, zscore
from itertools import combinations
#import nltk
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn

PATH = '/media/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/'
filename = 'Recap_Odeurs_Evaluations.xlsx'
df_save = 'distance_odors_all_pairs_meth=sum.csv'
cols_sel = ['Od_ID','Pleasantness','Familiarity','intensity','struct','type']

df = pd.read_excel(PATH+filename,sheet_name='Final')
df['Od_ID'] = [x[1:] for x in df['od_num']]
df['intensity'] = [0 if x == 'medium' else 1 for x  in df['strength']]
df['struct'] = [0 if x == 'single' else 1 for x  in df['structure']]
for col in np.setdiff1d(cols_sel,['Od_ID','type']):
    df[col+'_z'] = [(x - min(df[col]))/(max(df[col]-min(df[col]))) \
                        for x in df[col]] 
    
ncombs = len([c1 for c1,c2 in combinations(df['Od_ID'].values,2)])
od_vec, dist_vec = [], np.zeros((ncombs,5))
i=0
for c1, c2 in combinations(df['Od_ID'],2):
    r1 = df.loc[df['Od_ID']==c1]
    r2 = df.loc[df['Od_ID']==c2]
    r3 = []
    for col in ['Od_ID','Pleasantness_z','Familiarity_z','intensity_z','struct_z','type']:
        if col == 'Od_ID':
            od_vec.append(r1[col].values[0]+'_'+r2[col].values[0])
        elif col == 'type':
            syn1 = wn.synsets(r1[col].values[0])[0]
            syn2 = wn.synsets(r2[col].values[0])[0]
            r3.append(np.round(syn1.wup_similarity(syn2),2))
        else:
            r3.append(np.round(np.linalg.norm(r1[col].values[0]-r2[col].values[0]),2))
    dist_vec[i] += r3
    i += 1

data = np.concatenate((np.array(od_vec)[:,np.newaxis],dist_vec),axis=1)
cols = ['Pl_d','Fam_d','Int_d','Stru_d','Type_d']
df = pd.DataFrame(data,columns=['od_pairs','Pl_d','Fam_d','Int_d','Stru_d','Type_d'])
for c in cols:
    df[c] = df[c].astype(float)
df['dist_sum'] = df[cols].sum(axis=1)
df.to_csv(PATH+df_save,index=False)    

### create distance file for every patient (different odor pairs)
- olfactory distance
- spatial distance between circles
- temporal distance between explorations (Day2 = time + 24h)
- respiration difference (volume of the first inspiration) only for Encoding analyses
- memory distance (episodic memory perf)

In [2]:
def compute_distance(df,r1,r2,col0,col1):
    xy1 = np.array([df[col0].iloc[r1].values[0],df[col1].iloc[r1].values[0]])
    xy2 = np.array([df[col0].iloc[r2].values[0],df[col1].iloc[r2].values[0]])
    dist = np.round(np.linalg.norm(xy1-xy2),2)
    return dist

In [6]:
from codes import num_odor_to_ref
from utils import pos_cirles, odor_su_score
import numpy as np
import pandas as pd
from itertools import combinations

subjects = ['CHAF','FERJ','LEFC','PIRJ','SEMC','VACJ']
PATH = '/media/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/'
df_perf = 'encoding_individual_results.xls'
df_od_name = 'distance_odors_all_pairs_meth=sum.csv'
df_save = 'distance_dims=all_odors=all_su={}_norm={}.csv'
cols_s = ['trial_time','odor_num','encoding_day','0_insp_V','0_exp_V']

for su in subjects:
    df_E = pd.read_excel(PATH+df_perf,sheet_name=su)
    df_E['0_insp_V'] = np.abs(df_E['total_encodage_0_insp_volume'])
    df_E['0_exp_V'] = df_E['total_encodage_0_exp_volume']*-1
    df_E = df_E[cols_s]
    df_E = df_E.fillna(df_E.mean())
    df_od = pd.read_csv(PATH+df_od_name)

    od_pairs, od_dist, temp_dist, resp_dist = [], [], [], []
    spa_dist, rich_dist = [], []
    for r1, r2 in combinations(df_E.index,2):
        o1 = str(df_E['odor_num'].iloc[r1])
        o2 = str(df_E['odor_num'].iloc[r2])
        p1 = 'P'+str(num_odor_to_ref[su][int(o1)][1])
        p2 = 'P'+str(num_odor_to_ref[su][int(o2)][1])

        if o1 != o2:
            comb, comb_inv = o1+'_'+o2, o2+'_'+o1
            od_pairs.append(comb)
            od_dist.append(df_od['dist_sum'].loc[df_od['od_pairs'].isin([comb,comb_inv])].values[0])
            temp_dist.append(compute_distance(df_E,r1,r2,
                                              col0=['trial_time'],col1=['encoding_day']))
            resp_dist.append(compute_distance(df_E,r1,r2,
                                              col0=['0_insp_V'],col1=['0_exp_V']))
            coord1, coord2 = np.array(pos_cirles[p1]), np.array(pos_cirles[p2])
            spa_dist.append(np.round(np.linalg.norm(coord1-coord2),2))
            score1 = np.array(odor_su_score[su][o1])
            score2 = np.array(odor_su_score[su][o2])
            rich_dist.append(np.round(np.linalg.norm(coord1-coord2),2))
    data = np.concatenate((np.array(od_pairs)[:,np.newaxis], 
                           np.array(od_dist)[:,np.newaxis], 
                           np.array(temp_dist)[:,np.newaxis], 
                           np.array(resp_dist)[:,np.newaxis], 
                           np.array(spa_dist)[:,np.newaxis],
                           np.array(rich_dist)[:,np.newaxis]),axis=1)
    df_dist = pd.DataFrame(data, columns=['od_pairs','od_dist','temp_dist',
                                         'resp_dist','spa_dist','rich_dist'])
    df_dist.to_csv(PATH+df_save.format(su,'False',index=False))
    #rescale all distances to be between 0 and 1
    for c in ['od_dist','temp_dist','resp_dist','spa_dist','rich_dist']:
        df_dist[c] = df_dist[c].astype(float)
        df_dist[c] = [(x - min(df_dist[c]))/(max(df_dist[c]-min(df_dist[c]))) \
                        for x in df_dist[c]]
    df_dist.to_csv(PATH+df_save.format(su,'True'),index=False)